In [ ]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [ ]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [10]:
pop=load_to_df('상권-추정유동인구')
hl_pop=load_to_df('상권배후지-추정유동인구')

In [11]:
weekend = {'여성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
         '남성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],}
weekends={'hl_여성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
         'hl_남성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],}
any_word = "시간대"

def delete_age_by_time(any_df, any_dict, any_word):
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_'+value+'유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).filter(like=any_word).columns
            any_df.drop(del_columns, axis=1, inplace=True)
                    
    return any_df

def delete_day1(any_df, any_dict):
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_주중_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

def delete_day2(any_df, any_dict):
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_주말_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

In [12]:
f_pop_a=delete_age_by_time(pop, weekend, any_word)
hl_f_pop=delete_age_by_time(hl_pop, weekends, any_word)

In [13]:
week = {'여성' : ['월요일','화요일','수요일','목요일','금요일'],
       '남성' : ['월요일','화요일','수요일','목요일','금요일'],}
week2 = {'hl_여성' : ['월요일','화요일','수요일','목요일','금요일'],
       'hl_남성' : ['월요일','화요일','수요일','목요일','금요일'],}
f_pop_b=delete_day1(f_pop_a, week)
hl_f_pop_b=delete_day1(hl_f_pop, week2)

In [14]:
weeks = {'여성' : ['토요일','일요일'], '남성':['토요일','일요일'],}
weeks2 = {'hl_여성' : ['토요일','일요일'], 'hl_남성':['토요일','일요일'],}
f_pop_c=delete_day2(f_pop_b, weeks)
hl_f_pop_c=delete_day2(hl_f_pop_b, weeks2)

In [15]:
def lioc(df):
    df['총_주중_유동인구수']=df.iloc[:,17:22].sum(axis=1)
    df['총_주말_유동인구수']=df.iloc[:,22:24].sum(axis=1)
    result=df.drop(df.iloc[:,17:24], axis=1)
    return result

def lioc2(df):
    df['hl_총_주중_유동인구수']=df.iloc[:,17:22].sum(axis=1)
    df['hl_총_주말_유동인구수']=df.iloc[:,22:24].sum(axis=1)
    result=df.drop(df.iloc[:,17:24], axis=1)
    return result

In [16]:
# f_pop_c['총_주중_유동인구수']=f_pop_c.iloc[:,17:22].sum(axis=1)
# f_pop_c['총_주말_유동인구수']=f_pop_c.iloc[:,22:24].sum(axis=1)
# f_pop_c.drop(f_pop_c.iloc[:,17:24], axis=1)

# hl_f_pop_c['총_주중_유동인구수']=hl_f_pop_c.iloc[:,17:22].sum(axis=1)
# hl_f_pop_c['총_주말_유동인구수']=hl_f_pop_c.iloc[:,22:24].sum(axis=1)
# hl_f_pop_c.drop(hl_f_pop_c.iloc[:,17:24], axis=1).info()

In [17]:
f_pop_d=lioc(f_pop_c)
hl_f_pop_d=lioc2(hl_f_pop_c)

In [18]:
def multi_merge(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', on=['기준_년_분기_코드','상권_코드'], sort=False)
    all_df = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df.dropna(axis=0)

In [19]:
flow_pop=multi_merge(f_pop_d, hl_f_pop_d)  # merging('상권-추정유동인구', '상권배후지-추정유동인구')

In [20]:
# delete_word_columns_1 = f_pop_a.filter(like='대_').columns
# f_pop_a.drop(delete_word_columns_1, axis=1, inplace=True)

In [21]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del a_df, b_df
    return all_df.dropna(axis=0)

In [22]:
# flow_pop=multi_columns_merging('상권-추정유동인구', '상권배후지-추정유동인구')
stay_pop=multi_columns_merging('상권-상주인구', '상권배후지-상주인구')
office_pop=multi_columns_merging('상권-직장인구','상권배후지-직장인구')

In [23]:
ch_df=load_to_df('상권-상권변화지표')
ch_19_df=ch_df[ch_df['기준_년_분기_코드'].str.contains('2019_1|2019_2')]
ch_19_name=ch_19_df[['기준_년_분기_코드','상권_코드','상권_변화_지표']]

In [24]:
ch_19_df[['기준_년_분기_코드','상권_코드','상권_변화_지표']].head()

,기준_년_분기_코드,상권_코드,상권_변화_지표
4488,2019_2,1001496,HH
4489,2019_2,1001495,LL
4490,2019_2,1001494,HH
4491,2019_2,1001493,HH
4492,2019_2,1001492,HH


In [25]:
# flow_pop.shape , stay_pop.shape, office_pop.shape

In [26]:
people_a = multi_merge(flow_pop,stay_pop)
people = multi_merge(people_a,office_pop)   #유동,상주,직장인구 총 데이터 merge 

In [27]:
f_pop_ch=multi_merge(flow_pop, ch_19_name)   #유동인구 + 지표
s_pop_ch=multi_merge(stay_pop, ch_19_name)   #상주인구 + 지표
o_pop_ch=multi_merge(office_pop, ch_19_name) #직장인구 + 지표
total_people=multi_merge(people, ch_19_name)

In [28]:
pd.get_dummies(f_pop_ch, columns=['상권_변화_지표'], drop_first=False).head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,여성_주중_유동인구수,남성_주중_유동인구수,여성_주말_유동인구수,남성_주말_유동인구수,총_주중_유동인구수,...,hl_여성_주중_유동인구수,hl_남성_주중_유동인구수,hl_여성_주말_유동인구수,hl_남성_주말_유동인구수,hl_총_주중_유동인구수,hl_총_주말_유동인구수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,2019_1,1000001,118266,61268,56997,9103,10008,7228,6575,86815,...,35595.0,43907.0,26036.0,22734.0,359301.0,112607.0,0,0,1,0
1,2019_1,1000002,246619,147358,99261,14616,20191,16578,30739,156385,...,152822.0,196273.0,126465.0,211714.0,1499105.0,686921.0,0,1,0,0
2,2019_1,1000003,144480,80073,64407,10823,13782,7294,8392,107665,...,135762.0,176067.0,90027.0,104445.0,1340682.0,477893.0,1,0,0,0
3,2019_1,1000004,322387,155635,166753,23108,22411,18610,16982,248361,...,88426.0,82057.0,76509.0,68608.0,914237.0,294728.0,0,0,1,0
4,2019_1,1000005,23622,10781,12848,2053,1638,2438,2079,14580,...,4058.0,3445.0,4632.0,3894.0,30616.0,16966.0,0,0,0,1


In [29]:
pd.get_dummies(s_pop_ch, columns=['상권_변화_지표'], drop_first=False).head()

,기준_년_분기_코드,상권_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,hl_총_가구_수,hl_아파트_가구_수,hl_비_아파트_가구_수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,2019_2,1001010,3088,1526,1562,1601,0,1601,28320.0,13999.0,14321.0,13046.0,1888.0,11158.0,0,0,0,1
1,2019_2,1001009,4032,2056,1976,2067,39,2028,22242.0,10954.0,11288.0,10288.0,1770.0,8518.0,0,1,0,0
2,2019_2,1001008,2323,1114,1209,1261,0,1261,16300.0,8141.0,8159.0,8379.0,265.0,8114.0,0,0,0,1
3,2019_2,1001007,2208,1105,1103,980,0,980,5262.0,2676.0,2586.0,2175.0,182.0,1993.0,0,0,1,0
4,2019_2,1001006,4523,2227,2296,1738,531,1207,21885.0,10817.0,11068.0,8598.0,3081.0,5517.0,0,0,1,0


In [30]:
pd.get_dummies(o_pop_ch, columns=['상권_변화_지표'], drop_first=False).head()

,기준_년_분기_코드,상권_코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,2019_2,1001010,231,73,158,2935.0,1774.0,1161.0,0,0,0,1
1,2019_2,1001009,543,378,165,6413.0,3834.0,2579.0,0,1,0,0
2,2019_2,1001008,149,95,54,4362.0,2489.0,1873.0,0,0,0,1
3,2019_2,1001007,462,305,157,1171.0,816.0,355.0,0,0,1,0
4,2019_2,1001006,504,261,243,1996.0,1108.0,888.0,0,0,1,0


In [31]:
pd.get_dummies(total_people, columns=['상권_변화_지표'], drop_first=False).head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,여성_주중_유동인구수,남성_주중_유동인구수,여성_주말_유동인구수,남성_주말_유동인구수,총_주중_유동인구수,...,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,2019_1,1000001,118266,61268,56997,9103,10008,7228,6575,86815,...,348,142,206,24345.0,19877.0,4468.0,0,0,1,0
1,2019_1,1000002,246619,147358,99261,14616,20191,16578,30739,156385,...,486,254,232,13270.0,6153.0,7117.0,0,1,0,0
2,2019_1,1000003,144480,80073,64407,10823,13782,7294,8392,107665,...,4822,2188,2634,44890.0,32318.0,12572.0,1,0,0,0
3,2019_1,1000004,322387,155635,166753,23108,22411,18610,16982,248361,...,104,58,46,2196.0,1194.0,1002.0,0,0,1,0
4,2019_1,1000005,23622,10781,12848,2053,1638,2438,2079,14580,...,1080,692,388,1820.0,994.0,826.0,0,0,0,1


In [32]:
# 다중 컬럼에서 부분 추출 할때 사용
col_word=['상권_코드','총']    #컬럼 단어 입력하기
def col_value_filter(total_df,col_word):
    result=total_df.filter(regex='|'.join(col_word))
    return result

In [33]:
targets=['기준_년_분기_코드', '상권_코드', 
         '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수',
         '총_상주인구_수', '남성_상주인구_수', '여성_상주인구_수',
         '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수']
# total_pop=people_b.filter(regex='|'.join(targets)) #총인구 남자수,여자수 merge 한 값

In [34]:
total_people=col_value_filter(people,targets)
total_people.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,955,5455.0,2544.0,2911.0,348,142,206,24345.0,19877.0,4468.0
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,458,11047.0,5624.0,5423.0,486,254,232,13270.0,6153.0,7117.0
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,532,3512.0,2033.0,1479.0,4822,2188,2634,44890.0,32318.0,12572.0
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,742,9137.0,4521.0,4616.0,104,58,46,2196.0,1194.0,1002.0
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,868,6102.0,2958.0,3144.0,1080,692,388,1820.0,994.0,826.0


In [35]:
total_pop_ch=multi_merge(total_people, ch_19_df)   #인구수+지표 총 합한거
total_pop_ch.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,...,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,...,206,24345.0,19877.0,4468.0,LH,상권확장,88,57,107,53
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,...,232,13270.0,6153.0,7117.0,HL,상권축소,116,47,107,53
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,...,2634,44890.0,32318.0,12572.0,HH,정체,113,56,107,53
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,...,46,2196.0,1194.0,1002.0,LH,상권확장,88,57,107,53
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,...,388,1820.0,994.0,826.0,LL,다이나믹,85,51,107,53


In [36]:
# targets2=['상권코드','인구','지표']
# col_value_filter(total_pop_ch,targets2)
t_pop_ch=total_pop_ch.iloc[:,:21]

In [37]:
t_pop_ch.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,...,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,...,5455.0,2544.0,2911.0,348,142,206,24345.0,19877.0,4468.0,LH
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,...,11047.0,5624.0,5423.0,486,254,232,13270.0,6153.0,7117.0,HL
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,...,3512.0,2033.0,1479.0,4822,2188,2634,44890.0,32318.0,12572.0,HH
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,...,9137.0,4521.0,4616.0,104,58,46,2196.0,1194.0,1002.0,LH
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,...,6102.0,2958.0,3144.0,1080,692,388,1820.0,994.0,826.0,LL


In [38]:
t_pop_ch['상권_변화_지표'].unique()

array(['LH', 'HL', 'HH', 'LL'], dtype=object)

In [39]:
pd.get_dummies(t_pop_ch, columns=['상권_변화_지표'], drop_first=False)

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,...,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,...,348,142,206,24345.0,19877.0,4468.0,0,0,1,0
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,...,486,254,232,13270.0,6153.0,7117.0,0,1,0,0
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,...,4822,2188,2634,44890.0,32318.0,12572.0,1,0,0,0
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,...,104,58,46,2196.0,1194.0,1002.0,0,0,1,0
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,...,1080,692,388,1820.0,994.0,826.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_2,1001006,309262,149761,159498,1742709.0,852106.0,890599.0,4523,2227,...,504,261,243,1996.0,1108.0,888.0,0,0,1,0
2016,2019_2,1001007,55873,37566,18307,197220.0,129539.0,67683.0,2208,1105,...,462,305,157,1171.0,816.0,355.0,0,0,1,0
2017,2019_2,1001008,390072,189332,200742,3983885.0,1862103.0,2121778.0,2323,1114,...,149,95,54,4362.0,2489.0,1873.0,0,0,0,1
2018,2019_2,1001009,738256,369072,369178,3786483.0,1819086.0,1967396.0,4032,2056,...,543,378,165,6413.0,3834.0,2579.0,0,1,0,0
